# Searching for objects in mock images from the ArcsFinding Challenge

What I we are going to do now is to build the pipeline from some methods explored in the previous notebook ``segmenting image``.

The pipeline should get one image --monochromatic image-- and output (*i*) a table (~pandas.DataFrame) with morphological quantidies along the lines and the statistical measurements as columns, (*ii*) a panel with figures of each step of the processing (including the input image). The features table should also be written as a CSV file.

The input files are named as "``imageEUC_VIS-``*id*``.fits``".
"*id*" is a number, unique to each image.

The name of the output files should be "*id*``-featureMatrix.csv``" and "*id*``-partialOuts.png``".

The pipeline goes like:
* image normalization
* pre-processing
 * smoothing
* segmentation
 * background threshold
 * contours definition
 * local maxima
 * regions mask
 * objects identification
* post-processing
 * features extraction
* output

In [1]:
class PipelineBase:

    _inputFileFormat = 'imageEUC_VIS-{0}.fits'
    _outputTableFileFormat = 'featuresMatrix-{0}.csv'
    _outputPlotFileFormat = 'partialOuts-{0}.png'
    
    def __init__(self,id):
        self.id = id

    def read_image(self):
        assert False, "Base class, method not implemented"
        
    def normalize(self):
        assert False, "Base class, method not implemented"
        
    def smooth(self):
        assert False, "Base class, method not implemented"

    def estimate_threshold(self):
        assert False, "Base class, method not implemented"
    
    def find_maxima(self):
        assert False, "Base class, method not implemented"
    
    def find_countours(self):
        assert False, "Base class, method not implemented"
    
    def define_objects(self):
        assert False, "Base class, method not implemented"
    
    def compute_features(self):
        assert False, "Base class, method not implemented"
    
    def write_table(self):
        assert False, "Base class, method not implemented"
    
    def write_plot(self):
        assert False, "Base class, method not implemented"
    

    
def normalize(img,unit=1):
    """
    Normalize 'img' to 'unit'
    """
    
    return unit * (img - img.min())/(img.max() - img.min())

def gaussian(img, sigma=[3,3]):
    """
    Simple gaussian filter
    
    Input:
     - img <ndarray>
     - sigma <[int,int]> : sigma window
    
    Output:
     - <ndarray>
    
    ---
    """
    import scipy.ndimage as ndi
    return ndi.gaussian_filter(img,sigma)

def histmax(img):
    """
    Maximum histogram value for threshold estimation
    """
    import numpy as np
    nbins=1000
    imhist,bins = np.histogram(img.flatten(),nbins,normed=True);

    return bins[np.argmax(imhist)] + np.std(img)

def mask_contours(contours, img_shape):
    import numpy as np
    
    def define_regions(contours):
        '''
        Return a list of regions (~matplotlib.path.Path), for each entry in 'country_map'

        'country_map' is expected to provide columns 'lons' and 'lats',
        providing lists of coordinates defining the respective country
        as polygon(s). If 'country_map' has multiple lines, multiple
        "Paths" will be created (for each line/polygon).
        '''
        from matplotlib.path import Path
        import numpy as np
        regions_path = []
        for n, contour in enumerate(contours):
            regions_path.append( Path( contour ) )
        return regions_path

    # Create vertex coordinates for each grid cell...
    # (<0,0> is at the top left of the grid in this system)
    ny,nx = img_shape
    x, y = np.meshgrid(np.arange(nx), np.arange(ny))
    x, y = x.flatten(), y.flatten()

    grid = np.vstack((y,x)).T

    mask = np.zeros(img_shape).astype(bool)
    paths = define_regions(contours)
    for path in paths:
        _mask = path.contains_points(grid)
        mask = mask + _mask.reshape((ny,nx))
    return mask

def region_growing(img,x,y,thresh):
    """
    Segment using a Region Growing algorithm
    
    Input:
     - img  ndarray : Image array (ndim=2)
     - x        int : Seed x position
     - y        int : Seed y position
     - thresh float : Threshold value for limiting the grow
    
    Output:
     - region  : Region grown around given 'x,y'
    
    ---
    """
    from scipy import ndimage as ndi
    import numpy as np
    
    x_o = x
    y_o = y
    flag = True

    # Initialize region with the seed point
    region = np.zeros(img.shape,dtype=np.bool)
    reg_old = (region==flag)

    if (img[y_o,x_o] < thresh):
        return region
    
    region[y_o,x_o] = flag
    reg_cur = (region==flag)

    # For future (loop) morphological operations...
    strct_elem = ndi.generate_binary_structure(2,2)

    # While region stills changes (grow), do...
    while not np.all(region == reg_old):
        
        reg_old = region.copy()

        # Define pixel neighbours
        tmp = ndi.binary_dilation(region,strct_elem, 1)
        neigbors = tmp - region
        inds = np.where(neigbors)

        # Check for the new neighbors; do they fullfil requirements?
        #region[np.where(region[inds]>=thresh)] = flag
        for y_i,x_i in zip(*inds):
            if (img[y_i,x_i] >= thresh):
                region[y_i,x_i] = flag

    return region

def objects_properties(img_segm, img_smooth):
    import numpy as np
    from pandas import DataFrame
    from skimage.measure import regionprops
    reg_props = regionprops(img_segm,img_smooth)

    d_props = {}
    l_props = []
    d2_props = {}
    l2_props = []
    for props in reg_props:
        label = props.label
        vals = []
        vals2 = []
        for prop in props:
            val = props[prop]
            if np.isscalar(val):
                vals.append(val)
                l_props.append(prop)
            else:
                vals2.append(val)
                l2_props.append(prop)
        d_props[label] = vals
        d2_props[label] = vals2
    # 'df' contain morphological parameters from each identified object
    index = l_props[:len(vals)]
    df = DataFrame(d_props,index=index).transpose()
    return df

def plot_panelImages(img,img_smooth,img_maxima,contours,img_segm):
    from matplotlib import pyplot as plt

    num_panels = 4
    fig, axes = plt.subplots(ncols=num_panels, figsize=(12, 3), sharex=True, sharey=True,
                             subplot_kw={'adjustable': 'box-forced'})
    ax = axes.ravel()

    _ = ax[0].imshow(img, cmap=plt.cm.gray, interpolation='nearest')
    ax[0].set_title('Image (original)')

    _ = ax[1].imshow(-img_smooth, cmap=plt.cm.gray, interpolation='nearest')
    ax[1].set_title('Image smooth')

    ax[2].imshow(img_maxima, cmap=plt.cm.gray)
    for n, contour in enumerate(contours):
        ax[2].plot(contour[:, 1], contour[:, 0], linewidth=2, color='red')
    ax[2].set_title('Contours and maxima')
    ax[2].set_xticks([])
    ax[2].set_yticks([])

    im = ax[3].imshow(img_segm, cmap=plt.cm.spectral, interpolation='nearest')
    ax[3].set_title('Segmented objects')

    cax = fig.add_axes([0.91, 0.2, 0.01, 0.6])
    import numpy as np
    vals = np.unique(img_segm)
    fig.colorbar(im, cax=cax, ticks=vals)

    for a in ax:
        a.set_axis_off()
    
    return plt


In [2]:
class Pipeline(PipelineBase):
    
    def read_image(self,dir=None):
        from astropy.io import fits
        filename = self._inputFileFormat.format(self.id)
        if dir != None:
            from os import path
            filename = path.join(dir,filename)
        
        img = fits.open(filename)[0].data
        return img
    
    def normalize(self,img):
        return normalize(img)
    
    def smooth(self,img):
        return gaussian(img,[2,2])
    
    def estimate_threshold(self,img):
        return histmax(img)
    
    def find_contours(self,img,thresh):
        from skimage import measure
        contours = measure.find_contours(img,thresh)
        return contours
    
    def find_maxima(self,img):
        from skimage.feature import peak_local_max
        img_local_maxi = peak_local_max(img, min_distance=2, exclude_border=False, indices=False)
        return img_local_maxi
    
    def define_objects(self,img,thresh,maxima):
        import numpy as np
        if maxima.shape == img.shape:
            _lmi = np.where(maxima)
            local_maxi_yx = zip(_lmi[0],_lmi[1])
        else:
            local_maxi_yx = maxima
        img_reg = np.zeros(img.shape,dtype=int)
        for n,yx in enumerate(local_maxi_yx):
            y,x = yx
            reg = region_growing(img,x,y,thresh)#+0.1)
            _img = reg.astype(int) * n
            _idx = np.where(reg)
            img_reg[_idx] = _img[_idx]
        return img_reg

    def compute_features(self, img_segm, img_smooth):
        import pandas
        try:
            tab_objects = objects_properties(img_segm, img_smooth)
            _features_matrix_T = tab_objects.describe()
            _skew_cols = tab_objects.skew()
            _kurt_cols = tab_objects.kurtosis()
            _df = pandas.DataFrame([_skew_cols,_kurt_cols],index=['skew','kurtosis'],columns=tab_objects.columns)
            features_matrix = pandas.concat([_features_matrix_T,_df]).transpose()
        except:
            features_matrix = None

        return features_matrix
    
    def write_table(self,df,dir='output'):
        import os
#         _dir = os.path.join(dir,str(self.id))
        _dir = dir
        mkdir(_dir)
        filename = self._outputTableFileFormat.format(self.id)
        filename = os.path.join(_dir,filename)
        df.to_csv(filename)
        
    def write_plot(self,plot,dir='output'):
        import os
#         _dir = os.path.join(dir,str(self.id))
        _dir = dir
        mkdir(_dir)
        filename = self._outputPlotFileFormat.format(self.id)
        filename = os.path.join(_dir,filename)
        plot.savefig(filename)
        
def mkdir(path):
    import os
    _pathblocks = path.split(os.path.sep)
    _path = ''
    for _dir_ in _pathblocks:
        _path = os.path.join(_path,_dir_)
        if not os.path.exists(_path):
            os.mkdir(_path)
    return os.path.join(_path)

In [3]:
def run_pipeline(id,input_dir='data/'):
    ppline = Pipeline(id)

    # read data
    img = ppline.read_image(dir=input_dir)

    # normalize image
    img = ppline.normalize(img)

    # smooth the image
    img_smooth = ppline.smooth(img)

    # get the background threshold
    back_thresh = ppline.estimate_threshold(img)

    # define the countours
    contours = ppline.find_contours(img_smooth, back_thresh)

    # use contours as the valid/candidate regions
    _mask = mask_contours(contours, img.shape)

    # find the local maxima
    img_maxima = ppline.find_maxima(img_smooth * _mask)

    # better define the objects
    img_segm = ppline.define_objects(img, back_thresh, img_maxima)

    # generate the image processing plots
    plot = plot_panelImages(img,img_smooth,img_maxima,contours,img_segm)
    ppline.write_plot(plot)
    
    # compute the features
    features_matrix = ppline.compute_features(img_segm, img_smooth)
    if not features_matrix is None:
        ppline.write_table(features_matrix)

    return features_matrix


# read/define id
id = 100003

features_matrix = run_pipeline(id,'data/')

/home/chbrandt/.conda/envs/booq-dev/lib/python2.7/site-packages/ipykernel/__main__.py:151: DeprecationWarning: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.


In [4]:
features_matrix

,count,mean,std,min,25%,50%,75%,max,skew,kurtosis
area,6.0,172.166667,169.641288,13.000000,53.250000,103.500000,291.750000,422.000000,0.812830,-1.419060
convex_area,6.0,711.500000,1086.728071,13.000000,61.000000,163.000000,895.750000,2763.000000,1.805071,2.969080
eccentricity,6.0,0.656375,0.247219,0.329209,0.540171,0.593026,0.846596,0.969315,0.263787,-1.189019
equivalent_diameter,6.0,13.093467,7.571304,4.068429,8.223192,11.248391,19.012782,23.179885,0.380733,-1.657971
euler_number,6.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
extent,6.0,0.447165,0.301338,0.082005,0.189305,0.488044,0.664015,0.812500,-0.203864,-1.892205
filled_area,6.0,172.166667,169.641288,13.000000,53.250000,103.500000,291.750000,422.000000,0.812830,-1.419060
label,6.0,3.666667,2.160247,1.000000,2.250000,3.500000,4.750000,7.000000,0.462910,-0.300000
major_axis_length,6.0,38.370325,44.316619,4.318667,8.900622,14.965102,63.585279,109.301946,1.122066,-0.671105
max_intensity,6.0,0.340745,0.124014,0.190913,0.284893,0.334849,0.348291,0.562848,1.142450,2.509521


## Running the pipeline for all images

We have 20k images of this kind, some contain the lensing effect, other not.

In this computer I have such data below `/home/chbrandt/fido/Data/GravLensChallenge/SpaceBasedTraining`.

The dataset is composed by a *master table* where we find general properties for each *image* -- for instance,
whether in that image there is a lens or not --, each image is referred by its *ID* number; `classifications.csv`
is this table's filename.
The image files are placed under `Data.0/Public/Band1/` and `Data.1/Public/Band1/` directories, they are *FITS*
files named as `imageEUC_VIS-`*id*`.fits`.


In [5]:
image_properties = features_matrix.index
image_properties

Index([u'area', u'convex_area', u'eccentricity', u'equivalent_diameter',
       u'euler_number', u'extent', u'filled_area', u'label',
       u'major_axis_length', u'max_intensity', u'mean_intensity',
       u'min_intensity', u'minor_axis_length', u'orientation', u'perimeter',
       u'solidity'],
      dtype='object')

In [6]:
import pandas
tab_master_class = pandas.read_csv('classifications.csv')

In [7]:
pandas.concat([tab_master_class.head(), tab_master_class.tail()])

,ID,is_lens,Einstein_area,numb_pix_lensed_image,flux_lensed_image_in_sigma
0,100000,1,8.633760e-10,171,195.429
1,100001,1,1.317890e-10,294,855.589
2,100002,1,4.877250e-12,140,486.113
3,100003,1,1.440160e-09,1500,10467.400
4,100004,0,2.197350e-11,0,0.000
19995,119995,0,1.041840e-09,0,0.000
19996,119996,1,4.850260e-10,0,0.000
19997,119997,0,1.395720e-11,0,0.000
19998,119998,1,4.251510e-10,1024,4566.210
19999,119999,1,4.514050e-12,262,1741.430


In [8]:
%%time 
all_props = {}
FAILED = []
for i,row in enumerate(tab_master_class.iterrows()):
    # for testing reasons:
    if i == 100:
        break

    index,serie = row

    id_column = 'ID'
    class_column = 'is_lens'
    
    _id     = int(serie[id_column])
    _isLens = bool(serie[class_column])
    
    _props = run_pipeline(_id,'datalinks/')
    if _props is None:
#         print "Processing of ID:{} image/properties went wrong; Skipping this one.".format(_id)
        FAILED.append(_id)
        continue
        
#     _props.fillna(-999,inplace=True)
    _props[class_column] = int(_isLens)
    del _isLens
    
    all_props[_id] = _props
    del _id
    del _props
    
import pandas
data_panel = pandas.Panel(all_props)

/home/chbrandt/.conda/envs/booq-dev/lib/python2.7/site-packages/ipykernel/__main__.py:151: DeprecationWarning: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.
/home/chbrandt/.conda/envs/booq-dev/lib/python2.7/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


CPU times: user 58.1 s, sys: 387 ms, total: 58.5 s
Wall time: 58.4 s


In [9]:
data_panel

<class 'pandas.core.panel.Panel'>
Dimensions: 69 (items) x 16 (major_axis) x 11 (minor_axis)
Items axis: 100000 to 100099
Major_axis axis: area to solidity
Minor_axis axis: count to is_lens

In [10]:
dpT = data_panel.transpose(1,0,2)

In [11]:
dpT.items

Index([u'area', u'convex_area', u'eccentricity', u'equivalent_diameter',
       u'euler_number', u'extent', u'filled_area', u'label',
       u'major_axis_length', u'max_intensity', u'mean_intensity',
       u'min_intensity', u'minor_axis_length', u'orientation', u'perimeter',
       u'solidity'],
      dtype='object')

In [12]:
dpT['area']

,count,mean,std,min,25%,50%,75%,max,skew,kurtosis,is_lens
100000,2.0,86.000000,98.994949,16.0,51.00,86.0,121.00,156.0,NaN,NaN,1.0
100001,1.0,589.000000,NaN,589.0,589.00,589.0,589.00,589.0,NaN,NaN,1.0
100002,1.0,171.000000,NaN,171.0,171.00,171.0,171.00,171.0,NaN,NaN,1.0
100003,6.0,172.166667,169.641288,13.0,53.25,103.5,291.75,422.0,0.812830,-1.419060,1.0
100004,2.0,134.000000,134.350288,39.0,86.50,134.0,181.50,229.0,NaN,NaN,0.0
100007,2.0,258.000000,336.582828,20.0,139.00,258.0,377.00,496.0,NaN,NaN,1.0
100008,1.0,11.000000,NaN,11.0,11.00,11.0,11.00,11.0,NaN,NaN,1.0
100009,2.0,143.500000,161.927453,29.0,86.25,143.5,200.75,258.0,NaN,NaN,0.0
100010,1.0,238.000000,NaN,238.0,238.00,238.0,238.00,238.0,NaN,NaN,1.0
100011,3.0,59.000000,46.227697,6.0,43.00,80.0,85.50,91.0,-1.622374,NaN,1.0


In [13]:
len(FAILED)

31

In [14]:
FAILED

[100005,
 100006,
 100015,
 100016,
 100017,
 100023,
 100025,
 100030,
 100037,
 100041,
 100044,
 100047,
 100055,
 100062,
 100066,
 100067,
 100068,
 100071,
 100074,
 100075,
 100076,
 100077,
 100080,
 100081,
 100083,
 100085,
 100089,
 100090,
 100091,
 100092,
 100094]